In [ ]:
#Importing Dependencies
import pandas as pd
import requests
import datetime 
import json

#Start Dates
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 12, 1)
delta = datetime.timedelta(days=25)

start_list =[]

while start_date <= end_date:
    start_list.append(start_date)
    start_date += delta

#End Dates
start_date = datetime.date(2021, 1, 26)
end_date = datetime.date(2021, 12, 1)
delta = datetime.timedelta(days=25)

end_list =[]

while start_date <= end_date:
    end_list.append(start_date)
    start_date += delta

#Converting lists to strings 
end_str_list = []
for i in range(0,len(end_list)):
    end_str_list.append(end_list[i].strftime('%Y%m%d'))
    
start_str_list =[]
for i in range(0,len(start_list)):
    start_str_list.append(start_list[i].strftime('%Y%m%d'))

# Code for 2021 data
start_list = pd.date_range(start='1/1/2021', end ='1/1/2022', freq='MS')

start_str_list = []
end_str_list = []
for date in start_list:
    start_str_list.append(date.strftime('%Y%m%d'))
    end_str_list.append(date.strftime('%Y%m%d'))

start_str_list = start_str_list[0:12]
end_str_list = end_str_list[1:13]

In [ ]:
# Code for 60 days' retro data
today = pd.to_datetime("today") + datetime.timedelta(days=1)  

start_str_list = []
end_str_list = []

retro_start = 60
retro_end = 30

# For start dates
for x in range(0,2):
    if retro_start >= 0:
        start = today - datetime.timedelta(days=retro_start)
        start_str_list.append(start.strftime('%Y%m%d'))
        retro_start -=30
        
    else:
        pass

# For end dates
for y in range(0,1):
    if retro_end > 0:
        end = today - datetime.timedelta(days=retro_end)
        end_str_list.append(end.strftime('%Y%m%d'))        
        retro_end -=30
        
    else:
        pass

end_str_list.append(today.strftime('%Y%m%d'))

In [ ]:
#List of endpoints for API
complete_endpoints = []
for i in range(0,len(start_str_list)):
    complete_endpoints.append(f"{start_str_list[i]}:{end_str_list[i]}")

In [ ]:
#Base URL setup
base_url = "https://www.ncdc.noaa.gov/swdiws/json/nx3tvs/"

In [ ]:
#List of all URLs
all_urls = []
for dates in complete_endpoints:
    complete_url = base_url + dates
    all_urls.append(complete_url)
#print(len(all_urls))

In [ ]:
#List/Variables setup
cell_type =[]
shape =[]
max_shear = []
wsr_id =[]
mxdv = []
cell_id =[]
ztime = []
azimuth = []
range_ = []
lat = []
lon = []

#url_counter = 1

# Start of Log
print("-----------------------------")
print("Beginning Data Retrieval - estimated time is <5min")
print("-----------------------------")

#----BEGIN Loop----
# Looping through urls and appending data to lists
for url in all_urls:
    try:
        response = requests.get(url).json()
        for i in range(len(response["result"])):
            cell_type.append(response["result"][i]["CELL_TYPE"])
            shape.append(response["result"][i]["SHAPE"])
            max_shear.append(response["result"][i]["MAX_SHEAR"])
            wsr_id.append(response["result"][i]["WSR_ID"])
            mxdv.append(response["result"][i]["MXDV"])
            cell_id.append(response["result"][i]["CELL_ID"])
            ztime.append(response["result"][i]["ZTIME"])
            azimuth.append(response["result"][i]["AZIMUTH"])
            range_.append(response["result"][i]["RANGE"])
            lon.append(response["result"][i]["SHAPE"].split()[1:][0].split('(')[1])
            lat.append(response["result"][i]["SHAPE"].split()[1:][1].split(')')[0])
            #print(f"Processing Record {i} of URL {url_counter}")
        #url_counter += 1
    
    except(KeyError, IndexError):
        print(f"URL {url} not found. Skipping...")

#----END Loop-----

# End of Log
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------") 

In [ ]:
#Creating dataframe of all records
dict = {"Cell_Type": cell_type,
        "Shape": shape,
        "Max_Shear": max_shear,
        "WSR_ID": wsr_id,
        "MXDV": mxdv,
        "Cell_ID": cell_id,
        "zTime": ztime,
        "Azimuth": azimuth,
        "Range": range_,
        "Lat": lat,
        "Lon": lon
       }

dict["zTime"][0]

In [ ]:
#Creating additional columns
# df['Date']=pd.to_datetime(df['zTime']).dt.date
# df['Time']=pd.to_datetime(df['zTime']).dt.time
# df['Year']=pd.to_datetime(df['zTime']).dt.year
# df['Month']=pd.to_datetime(df['zTime']).dt.month

date = []
time = []
year = []
month = []
for x in ztime:
    date.append(datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').date())
    time.append(datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').time())
    year.append(datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').year)
    month.append(datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').month)
dict['Date']= date
dict['Time']= time
dict['Year']= year
dict['Month']= month

In [ ]:
#Primary Key to id tornados
#df['PKID']=df['Cell_ID']+df['zTime'].astype(str)
#df.head()

pkid = []
for x in range(len(cell_id)):
    pkid.append(cell_id[x] + ztime[x])
dict['PKID'] = pkid


In [ ]:
# df2 = dict.sort_values(['Date','Time'], ascending = [True,True])
# df2

#Function to build geojson
def df_to_geojson(df, properties, lat='Lat', lon='Lon'):
    #Create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    #Loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        #Create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        #Fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        #For each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        #Add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

# Setting columns and running function
cols = ["Cell_Type", "Shape", "Max_Shear", 
        "WSR_ID", "MXDV", "Cell_ID", 
        "zTime", "Azimuth", "Range", "Lat", "Lon",
        "Date","Time","Year","Month","PKID"]
geojson = df_to_geojson(df, cols)

#Saving output into js file
output_filename = 'static/data/tornadoes.geojson'
with open(output_filename, 'w') as output_file:
    json.dump(geojson, output_file, indent=2, default=str)

In [ ]:
# df.to_json('static/data/tornadoes.json', orient = 'records', compression = 'infer', index = 'true')